# Conduct EDA on linked dataset and baseline model results

In [ ]:
import pandas as pd
import geopandas as gpd
from shapely import wkt

import seaborn as sns
import matplotlib.pyplot as plt
from matplotlib.colors import Normalize
from matplotlib.colorbar import ColorbarBase
from matplotlib.cm import ScalarMappable, get_cmap

from pathlib import Path

## Load data directory

In [ ]:
# linked data
DATA_DIR = Path("../../data/linked_data")

In [ ]:
GEOG_HEALTH = DATA_DIR / "geog_health_city_merged.csv"

MODEL_DF = DATA_DIR / "city_merged_w_outbreak_tag.csv"
ANALYZE_DF = DATA_DIR / "analyze_city_merged_w_outbreak_preds.csv"

DAGUPAN_DENGUE = DATA_DIR / "dagupan_dengue_city_merged_w_outbreak_tag.csv"
DAGUPAN_ROTAVIRUS = DATA_DIR / "dagupan_rotavirus_city_merged_w_outbreak_tag.csv"
MUNTI_DENGUE = DATA_DIR / "muntinlupa_dengue_city_merged_w_outbreak_tag.csv"
MUNTI_ROTAVIRUS = DATA_DIR / "muntinlupa_rotavirus_city_merged.csv"

## 1. What are the Top CDs cases /NCDs deaths? One bar graph CD, One bar graph NCD

**Type**: Bar graphs/Pie Charts for Rankings, lists, percentages

### Load city information and health cases

In [ ]:
geog_health_df = pd.read_csv(GEOG_HEALTH)
geog_health_df.head(3)

In [ ]:
geog_health_df["date"] = pd.to_datetime(geog_health_df["date"])

In [ ]:
geog_health_df = geog_health_df[geog_health_df["date"].dt.year >= 2020]

In [ ]:
AREA_DATE_COLS = [
    "adm3_en",
    "adm3_pcode",
    "date",
    "year",
    "week",
    "brgy_distance_to_coast",
    "brgy_is_coastal",
]
CASE_COLS = geog_health_df.columns[
    geog_health_df.columns.str.startswith("case")
].tolist()
DEATH_COLS = geog_health_df.columns[
    geog_health_df.columns.str.startswith("death")
].tolist()

### Disease utils

In [ ]:
def plot_diseases_bar(df, cols_list=CASE_COLS, category="CD", city=""):
    count_df = df[cols_list].sum().sort_values(ascending=True)
    count_df.plot(kind="barh")

    if city == "":
        # Set the title and labels
        plt.title(f"Total historical sum of {category}s")
        plt.xlabel("Diseases")
        plt.ylabel("Sum of Cases")

        plt.show()
    else:
        # Set the title and labels
        plt.title(f"Total historical sum of {category}s - {city}")
        plt.xlabel("Diseases")
        plt.ylabel("Sum of Cases")

        plt.show()

### Historical plot of CDs 

In [ ]:
# historically from 2003 to 2022
plot_diseases_bar(geog_health_df, cols_list=CASE_COLS, category="CD")

### Historical Plot of NCDs

In [ ]:
# historically from 2003 to 2022
plot_diseases_bar(geog_health_df, cols_list=DEATH_COLS, category="NCD")

### Check all historical CDs and NCDs across the 12 cities

In [ ]:
city_names = geog_health_df["adm3_en"].unique().tolist()
print(city_names)

In [ ]:
CASE_COLS

In [ ]:
for city in city_names:
    print(f"Plot overall diseases count for {city}")
    city_df = geog_health_df[geog_health_df["adm3_en"] == city]
    # city_df = city_df.drop(columns=['case_total_dengue'])
    plot_diseases_bar(city_df, cols_list=CASE_COLS, category="CD", city=city)
    plot_diseases_bar(city_df, cols_list=DEATH_COLS, category="NCD", city=city)

In [ ]:
geog_health_df.groupby("adm3_en")[CASE_COLS].sum()

In [ ]:
geog_health_df.groupby("adm3_en")[DEATH_COLS].sum()

### Year on year growth

## 2. What diseases are coastal communities most susceptible to?

> Removing this question as there are no unique diseases to the communities.

In [ ]:
geog_health_df["brgy_distance_to_coast"].describe()

In [ ]:
# coastal brgy
coastal_areas = geog_health_df[geog_health_df["brgy_distance_to_coast"] <= 1631]
coastal_areas["adm3_en"].unique()

Plot seems to be the same as the overall diseases.

> Can omit this plot

In [ ]:
coastal_cd_sum = coastal_areas[CASE_COLS].sum().sort_values(ascending=False)
coastal_cd_sum.plot(kind="bar")

# Set the title and labels
plt.title("Total historical sum of CDs")
plt.xlabel("Diseases")
plt.ylabel("Sum of Cases")

## Prepare barangay tables with geometries

In [ ]:
# open barangay geometries
brgy_df = pd.read_csv(DATA_DIR / "tables/brgy_geography.csv")
brgy_df = brgy_df.drop(
    columns=["Unnamed: 0", "freq", "date", "uuid", "brgy_total_area"]
)
brgy_df["geometry"] = brgy_df["geometry"].apply(wkt.loads)

brgy_gdf = gpd.GeoDataFrame(brgy_df, geometry="geometry")
brgy_gdf.head(3)

In [ ]:
# open full admin boundary table
loc_df = pd.read_csv(DATA_DIR / "tables/location.csv")
loc_df = loc_df.drop(columns=["Unnamed: 0", "uuid"])
loc_df.head(3)

In [ ]:
# merge location and geometry tables
brgy_gdf = loc_df.merge(brgy_gdf, how="left", on="adm4_pcode")
brgy_gdf = gpd.GeoDataFrame(brgy_gdf)
brgy_gdf.head(2)

# What other variables contribute to the cases?

-  Population, and other environment variables (hazard, land use, built up areas)
-  Plot isochrones, access to healthcare facilities (choropleth)
-  Plot wealth index


# 4. How is each city's access to healthcare facilities relative to their area?

## % reached in health facilities

In [ ]:
# open health access files
brgy_iso = pd.read_csv(DATA_DIR / "tables/mapbox_health_facility_brgy_isochrones.csv")
brgy_iso = brgy_iso.drop(columns=["Unnamed: 0", "uuid", "date", "freq"])
brgy_iso.head(3)

In [ ]:
# keep only necessary cols
pct_cols = [col for col in brgy_iso.columns if col.__contains__("_pct")]
brgy_iso = brgy_iso[["adm4_pcode"] + pct_cols]
brgy_iso.head(3)

In [ ]:
brgy_iso = brgy_iso.merge(
    brgy_gdf[["adm3_en", "adm4_pcode", "geometry"]], on="adm4_pcode", how="left"
)

In [ ]:
brgy_iso = gpd.GeoDataFrame(brgy_iso)

In [ ]:
health_facility_type = ["brgy_healthcenter", "hospital", "rhu"]
pct_5min = [col for col in brgy_iso.columns if col.endswith("_5min")]
pct_15min = [col for col in brgy_iso.columns if col.endswith("_15min")]
pct_30min = [col for col in brgy_iso.columns if col.endswith("_30min")]

In [ ]:
cities = brgy_gdf["adm3_en"].unique().tolist()
print(cities)

In [ ]:
def plot_health_access(iso_gdf, city, pct_cols):
    fig, axs = plt.subplots(1, 3, figsize=(14, 6))
    # Define the number of rows and columns for subplots

    city_data = iso_gdf[iso_gdf["adm3_en"] == city]

    # Define colormap and normalization
    cmap = get_cmap("YlGnBu")
    norm = Normalize(
        vmin=city_data[pct_cols].min().min(), vmax=city_data[pct_cols].max().max()
    )

    # Iterate over the columns and plot each one
    for ax, column in zip(axs, pct_cols):
        city_data.plot(column=column, ax=ax, legend=False, cmap=cmap, norm=norm)
        ax.set_title(f"{column}")
        ax.axis("off")  # Turn off the axis

    # Create a color bar at the bottom of the plots
    cax = fig.add_axes(
        [0.1, 0.08, 0.8, 0.03]
    )  # Position for the color bar [left, bottom, width, height]
    sm = ScalarMappable(norm=norm, cmap=cmap)
    sm._A = []  # This is necessary for ScalarMappable
    cb = ColorbarBase(cax, cmap=cmap, norm=norm, orientation="horizontal")
    cb.set_label("Data Scale")  # You can customize this label

    plt.tight_layout(
        rect=[0, 0.15, 1, 1]
    )  # Adjust the rect to leave space for the color bar
    plt.title(city)
    plt.show()

In [ ]:
for city in cities:
    plot_health_access(brgy_iso, city=city, pct_cols=pct_5min)

## Population Density

In [ ]:
def plot_pop_density(
    pop_df, pop_col, vmin, vmax, cmap="Reds", label="Population Density"
):
    # Ensure the data is grouped by the 'city' column
    # max_val = pop_df[pop_col].max()
    grouped = pop_df.groupby("adm3_en")

    # Number of cities
    num_cities = len(grouped)

    # Create subplots - adjust the number of rows and columns as needed
    fig, axs = plt.subplots(
        nrows=int(num_cities**0.5) + 1,
        ncols=int(num_cities**0.5) + 1,
        figsize=(15, 18),
    )
    axs = axs.flatten()  # Flatten the array of axes

    # Define colormap and normalization
    cmap = get_cmap(cmap)
    # norm = Normalize(vmin=pop_df[pop_col].min().min(), vmax=pop_df[pop_col].max())

    # Loop through each city and plot it
    for i, (city_name, data) in enumerate(grouped):
        ax = axs[i]
        data.plot(
            ax=ax,
            legend=True,
            column=pop_col,
            cmap=cmap,
            legend_kwds={
                "label": label,
                "orientation": "horizontal",
                "shrink": 0.8,
                "anchor": (0.5, 1),
            },
            vmin=vmin,
            vmax=vmax
            # vmax=pop_df[pop_col].describe(percentiles=[0.75, 0.9, 0.95])['95%']
        )  # Plot the data
        ax.set_title(f"{city_name}")
        ax.axis("off")  # Turn off axis

    # Hide any unused axes if there are any
    for j in range(i + 1, len(axs)):
        axs[j].set_visible(False)

    # Create a color bar at the bottom of the plots
    # cax = fig.add_axes([0.1, 0.08, 0.8, 0.03])  # Position for the color bar [left, bottom, width, height]
    # sm = ScalarMappable(norm=norm, cmap=cmap)
    # sm._A = []  # This is necessary for ScalarMappable
    # cb = ColorbarBase(cax, cmap=cmap, norm=norm, orientation='horizontal')
    # cb.set_label('Data Scale')  # You can customize this label

    plt.tight_layout()  # Adjust the rect to leave space for the color bar
    # Show the plot
    plt.show()

In [ ]:
population_df = pd.read_csv(DATA_DIR / "tables/worldpop_population.csv")
population_df = population_df.drop(columns=["Unnamed: 0", "uuid"])
population_df = population_df[
    ["adm4_pcode", "date", "pop_count_total", "pop_count_mean"]
]
population_df.head()

In [ ]:
population_df["date"] = pd.to_datetime(population_df["date"])
population_df = population_df[population_df["date"].dt.year == 2020]
population_df = population_df.merge(
    brgy_gdf[["adm3_en", "adm4_pcode", "geometry"]], on="adm4_pcode", how="left"
)
population_gdf = gpd.GeoDataFrame(population_df)
population_gdf

In [ ]:
plot_pop_density(population_gdf, "pop_count_mean", vmin=0, vmax=None)

## Plot Wealth Index

In [ ]:
rwi_df = pd.read_csv(DATA_DIR / "tables/tm_relative_wealth_index.csv")
rwi_df = rwi_df.drop(columns=["Unnamed: 0", "uuid", "date", "freq"])
# hazards_df = hazards_df[['adm4_pcode', 'date', 'pop_count_total', 'pop_count_mean']]
rwi_df.head()

In [ ]:
rwi_df = rwi_df.merge(
    brgy_gdf[["adm3_en", "adm4_pcode", "geometry"]], on="adm4_pcode", how="left"
)
rwi_gdf = gpd.GeoDataFrame(rwi_df)

In [ ]:
plot_pop_density(rwi_gdf, "rwi_mean", vmin=0, vmax=1, cmap="viridis", label="RWI")

## Plot health care facilities

In [ ]:
doh_poi_health = pd.read_csv(DATA_DIR / "tables/geoportal_doh_poi_health.csv")
doh_poi_health = doh_poi_health.drop(columns=["Unnamed: 0", "uuid"])
doh_poi_health.head(2)

In [ ]:
doh_poi_health = doh_poi_health[
    doh_poi_health["adm4_pcode"].isin(dagupan_brgys_list + munti_brgys_list)
]
doh_poi_health = doh_poi_health.merge(
    loc_df[["adm3_en", "adm4_pcode"]], how="left", on="adm4_pcode"
)
doh_poi_health = doh_poi_health.merge(
    brgy_gdf[["adm4_pcode", "geometry"]], how="left", on="adm4_pcode"
)
doh_poi_health.head(2)

In [ ]:
doh_poi_gdf = gpd.GeoDataFrame(doh_poi_health)
dagupan_doh_gdf = doh_poi_gdf[doh_poi_gdf["adm3_en"] == "Dagupan City"]
munti_doh_gdf = doh_poi_gdf[doh_poi_gdf["adm3_en"] == "City of Muntinlupa"]

In [ ]:
ax = dagupan_doh_gdf.plot(column="doh_pois_count", legend=True, cmap="OrRd", vmax=8)
ax.set_title("Dagupan Health POIs")
plt.show()

In [ ]:
ax = munti_doh_gdf.plot(column="doh_pois_count", legend=True, cmap="OrRd")
ax.set_title("Muntinlupa Health POIs")
plt.show()